In [1]:
import os, sys
import django
import pytz
from dateutil import parser
from django.utils.timezone import make_aware

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "tshilo_dikotla.settings"

# Connect to Django ORM
django.setup()
import uuid
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

try:
    td = Site.objects.get(id=40)
except Site.DoesNotExist:
    td = Site.objects.create(id=40, domain='td-live.bhp.org.bw', name='td-live.bhp.org.bw')
path = '/Users/coulsonkgathi/source/td_data_files/20190707/'

In [2]:
# Import Subject Death
from td_prn.models import MaternalDeathReport
from td_maternal.models import SubjectConsent
from edc_action_item.models import ActionItem
from pprint import pprint

fname = path + 'maternal_death_report.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sd_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    sd_data.append(data)

gaborone = Site.objects.get(id=40)
already_created = 0
count = 0


for sd in sd_data:
    del sd['maternal_visit_id']
    del sd['_maternal_visit_cache']
    del sd['cause_category_id']
    del sd['reason_hospitalized_id']
    del sd['medical_responsibility_id']
    del sd['cause_id']
    del sd['_state']
    del sd['export_change_type']
    del sd['export_uuid']
    del sd['exported']
    del sd['exported_datetime']
    sd.update(
        created=make_aware(parser.parse(sd.get('created'))),
        id=uuid.UUID(sd.get('id')),
        site=gaborone,
        cause='relatives_friends',
        cause_category='hiv_unrelated',
        reason_hospitalized='non-infectious reason for hospitalization, specify',
        medical_responsibility='doctor'
    )
    if sd['death_date']:
        sd.update(death_date=make_aware(parser.parse(sd.get('death_date'))).date())
    else:
        sd.update(death_date=None)
    if sd['illness_duration']:
        sd.update(illness_duration=int(float(sd['illness_duration'])))
    else:
        sd.update(illness_duration=None)
    
    if sd['days_hospitalized']:
        sd.update(days_hospitalized=int(float(sd['days_hospitalized'])))
    else:
        sd.update(days_hospitalized=None)
    
    if sd['modified']:
        sd.update(modified=make_aware(parser.parse(sd.get('modified'))))
    else:
        sd.update(modified=None)
    if sd['report_datetime']:
        sd.update(report_datetime=make_aware(parser.parse(sd.get('report_datetime'))))
    else:
        sd.update(report_datetime=None)
    try:
        MaternalDeathReport.objects.get(subject_identifier=sd.get('subject_identifier'))
    except MaternalDeathReport.DoesNotExist:
        death_report = MaternalDeathReport(**sd)
        death_report.save()
        count += 1
    else:
        already_created += 1
print(f'Total created death report {count}')
print(f'Already created {already_created}')

Total created death report 1
Already created 0


In [3]:
# Import Subject Death
from td_prn.models import InfantDeathReport
from td_maternal.models import SubjectConsent
from edc_action_item.models import ActionItem
from pprint import pprint

fname = path + 'infant_death_report.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sd_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    sd_data.append(data)

gaborone = Site.objects.get(id=40)
already_created = 0
count = 0

cause_category = {
    1: 'hiv_related',
    2: 'hiv_unrelated',
    3: 'study_drug',
    4: 'non_study_drug',
    5: 'trauma',
    6: 'no_info',
    7: 'OTHER'
}

reason_hospitalized_option = {
    1: 'respiratory illness(unspecified)',
    2: 'respiratory illness, cxr confirmed',
    3: 'respiratory illness, cxr confirmed, bacterial pathogen, specify',
    4: 'respiratory illness, cxr confirmed, tb or probable tb',
    5: 'diarrhea illness(unspecified)',
    6: 'diarrhea illness, viral or bacterial pathogen, specify',
    7: 'sepsis(unspecified)',
    8: 'sepsis, pathogen specified, specify',
    9: 'mengitis(unspecified)',
    10: 'mengitis, pathogen specified, specify',
    11: 'OTHER',
    12: 'non-infectious reason for hospitalization, specify'
}

medical_responsibility_option = {
    1: 'doctor',
    2: 'nurse',
    3: 'traditional',
    4: 'all',
    5: 'none'
}

cause_option = {
    1: 'autopsy',
    2: 'clinical_records',
    3: 'study_staff',
    4: 'health_care_provider',
    5: 'death_certificate',
    6: 'relatives_friends',
    7: 'obituary',
    8: 'pending_information',
    9: 'no_info',
    10: 'OTHER'
}


for sd in sd_data:
    del sd['infant_visit_id']
    del sd['_infant_visit_cache']
    if sd['cause_category_id']:
        sd.update(cause_category=cause_category.get(int(float(sd['cause_category_id']))))
    else:
        sd.update(cause_category=None)
    del sd['cause_category_id']
    if sd['reason_hospitalized_id']:
        sd.update(reason_hospitalized=reason_hospitalized_option.get(int(float(sd['reason_hospitalized_id']))))
    else:
        sd.update(reason_hospitalized=None)
    del sd['reason_hospitalized_id']
    if sd['medical_responsibility_id']:
        sd.update(medical_responsibility=medical_responsibility_option.get(int(float(sd['medical_responsibility_id']))))
    else:
        sd.update(medical_responsibility=None)
    del sd['medical_responsibility_id']
    if sd['cause_id']:
        sd.update(cause=cause_option.get(int(float(sd['cause_id']))))
    else:
        sd.update(cause=None)
    del sd['cause_id']
    del sd['_state']
    del sd['export_change_type']
    del sd['export_uuid']
    del sd['exported']
    del sd['exported_datetime']
    del sd['registered_subject_id']
    sd.update(
        created=make_aware(parser.parse(sd.get('created'))),
        id=uuid.UUID(sd.get('id')),
        site=gaborone,
    )
    if sd['death_date']:
        sd.update(death_date=make_aware(parser.parse(sd.get('death_date'))).date())
    else:
        sd.update(death_date=None)
    if sd['illness_duration']:
        sd.update(illness_duration=int(float(sd['illness_duration'])))
    else:
        sd.update(illness_duration=None)
    
    if sd['days_hospitalized']:
        sd.update(days_hospitalized=int(float(sd['days_hospitalized'])))
    else:
        sd.update(days_hospitalized=None)
    
    if sd['modified']:
        sd.update(modified=make_aware(parser.parse(sd.get('modified'))))
    else:
        sd.update(modified=None)
    if sd['report_datetime']:
        sd.update(report_datetime=make_aware(parser.parse(sd.get('report_datetime'))))
    else:
        sd.update(report_datetime=None)
    try:
        InfantDeathReport.objects.get(subject_identifier=sd.get('subject_identifier'))
    except InfantDeathReport.DoesNotExist:
        death_report = InfantDeathReport(**sd)
        death_report.save()
        count += 1
    else:
        already_created += 1
print(f'Total created death report {count}')
print(f'Already created {already_created}')

Total created death report 6
Already created 0
